In [1]:
# Experiments imports
from ternary_weight_mlp import *
import json
import mlflow

## XOR Experiment

In [2]:
# HYPERPARAMETERS
INIITIAL_LEARNING_RATE  = 0.001  # Starting learning rate
MAXIMUM_LEARNING_RATE = 0.5   # Maximum learning rate
EPOCHS = 50000
ALPHA = 0.5
LAMBDA_MAGNITUDE = 0.1
LAMBDA_POLARITY = 0.1
LAMBDA_INTEGERS = 0.001
LAMBDA_SPARSITY = 0.01
X_train, y_train, X_test, y_test = create_torch_XOR_dataset()

EVALUTION_RUNS = 10
THRESHOLD = 0.02

# Counter for 100% accuracy runs
perfect_accuracy_count = 0
evaluation_results = []

for _ in range(EVALUTION_RUNS):
    model_XOR = TwoLayerMLP().double()

    # define the loss
    loss_function = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model_XOR.parameters(), lr=INIITIAL_LEARNING_RATE, eps=1e-7)

    all_loss = train_with_rectified_L2(model_XOR, 
                                    loss_function, 
                                    optimizer, 
                                    X_train, 
                                    y_train,
                                    no_of_epochs=EPOCHS,
                                    ALPHA=ALPHA,
                                    LAMBDA_MAGNITUDE=LAMBDA_MAGNITUDE,
                                    LAMBDA_POLARITY=LAMBDA_POLARITY,
                                    LAMBDA_INTEGERS=LAMBDA_INTEGERS,
                                    initial_lr=INIITIAL_LEARNING_RATE,
                                    max_lr=MAXIMUM_LEARNING_RATE)

    # Evaluate accuracy
    inputs = [torch.tensor([[0.0, 0.0]], dtype=torch.float64), torch.tensor([[0.0, 1.0]], dtype=torch.float64), torch.tensor([[1.0, 0.0]], dtype=torch.float64), torch.tensor([[1.0, 1.0]], dtype=torch.float64)]
    expected_outputs = [0.0, 1.0, 1.0, 0.0]
    correct_predictions = 0

    for input_tensor, expected_output in zip(inputs, expected_outputs):
        model_output = model_XOR(input_tensor).item()
        if abs(model_output - expected_output) <= THRESHOLD:
            correct_predictions += 1

    accuracy = correct_predictions / len(expected_outputs)

    if accuracy == 1.0:
        perfect_accuracy_count += 1

    # Evaluate interpretability
    results = evaluate_logic_gates_in_network(model_XOR, 2)
    results['accuracy'] = accuracy

    evaluation_results.append(results)

# Save the results to a JSON file
with open('evaluation_results.json', 'w') as f:
    json.dump(evaluation_results, f, indent=4)

Optimal solution found
Optimal solution found


## 3 layer MLP experiment

In [3]:
# HYPERPARAMETERS
INIITIAL_LEARNING_RATE  = 0.001  # Starting learning rate
MAXIMUM_LEARNING_RATE = 0.5   # Maximum learning rate
EPOCHS = 50000
ALPHA = 0.5
LAMBDA_MAGNITUDE = 0.1
LAMBDA_POLARITY = 0.1
LAMBDA_INTEGERS = 0.001
LAMBDA_SPARSITY = 0.01
X_train, y_train, X_test, y_test = create_torch_A_XOR_B_AND_C_NAND_C_dataset()

EVALUTION_RUNS = 10
THRESHOLD = 0.02

# Counter for 100% accuracy runs
perfect_accuracy_count = 0
evaluation_results = []

for _ in range(EVALUTION_RUNS):
    model_3_layer_MLP = SparseThreeLayerMLP2().double()

    # define the loss
    loss_function = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model_3_layer_MLP.parameters(), lr=INIITIAL_LEARNING_RATE, eps=1e-7)

    all_loss = train_with_rectified_L2(model_3_layer_MLP, 
                                    loss_function, 
                                    optimizer, 
                                    X_train, 
                                    y_train,
                                    no_of_epochs=EPOCHS,
                                    ALPHA=ALPHA,
                                    LAMBDA_MAGNITUDE=LAMBDA_MAGNITUDE,
                                    LAMBDA_POLARITY=LAMBDA_POLARITY,
                                    LAMBDA_INTEGERS=LAMBDA_INTEGERS,
                                    initial_lr=INIITIAL_LEARNING_RATE,
                                    max_lr=MAXIMUM_LEARNING_RATE)

    # Evaluate accuracy for the function (A xor B) and not C
    inputs = [
        torch.tensor([[0.0, 0.0, 0.0, 0.0]], dtype=torch.float64),
        torch.tensor([[0.0, 0.0, 1.0, 1.0]], dtype=torch.float64),
        torch.tensor([[0.0, 1.0, 0.0, 0.0]], dtype=torch.float64),
        torch.tensor([[0.0, 1.0, 1.0, 1.0]], dtype=torch.float64),
        torch.tensor([[1.0, 0.0, 0.0, 0.0]], dtype=torch.float64),
        torch.tensor([[1.0, 0.0, 1.0, 1.0]], dtype=torch.float64),
        torch.tensor([[1.0, 1.0, 0.0, 0.0]], dtype=torch.float64),
        torch.tensor([[1.0, 1.0, 1.0, 0.0]], dtype=torch.float64)
    ]
    expected_outputs = [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
    correct_predictions = 0

    for input_tensor, expected_output in zip(inputs, expected_outputs):
        model_output = model_3_layer_MLP(input_tensor).item()
        if abs(model_output - expected_output) <= THRESHOLD:
            correct_predictions += 1

    accuracy = correct_predictions / len(expected_outputs)

    if accuracy > 0.9:
        perfect_accuracy_count += 1

    # Evaluate interpretability
    results = evaluate_logic_gates_in_network(model_3_layer_MLP, 2)
    results['accuracy'] = accuracy

    evaluation_results.append(results)

# Save the results to a JSON file
with open('evaluation_results.json', 'w') as f:
    json.dump(evaluation_results, f, indent=4)

Optimal solution found


In [11]:
import json

# Load the JSON data from a file
def load_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

# Calculate the average accuracy and percentage
def calculate_averages(data):
    total_accuracy = 0
    total_percentage = 0
    count = len(data)
    
    for record in data:
        total_accuracy += record["accuracy"]
        total_percentage += record["percentage"]
    
    average_accuracy = total_accuracy / count if count != 0 else 0
    average_percentage = total_percentage / count if count != 0 else 0
    
    return average_accuracy, average_percentage


filename = 'experiment_results/evaluation_results_3layer_MLP.json'  # Replace with your actual JSON file name
data = load_json(filename)
average_accuracy, average_percentage = calculate_averages(data)

print(f"Average Accuracy: {average_accuracy}")
print(f"Average Percentage: {average_percentage}")

Average Accuracy: 0.675
Average Percentage: 84.0


### 8 Neuron MLP experiment

In [2]:
# HYPERPARAMETERS
INIITIAL_LEARNING_RATE  = 0.005  # Starting learning rate
MAXIMUM_LEARNING_RATE = 0.5   # Maximum learning rate
EPOCHS = 100000
ALPHA = 0.5
LAMBDA_MAGNITUDE = 0.1
LAMBDA_POLARITY = 0.1
LAMBDA_INTEGERS = 0.001
LAMBDA_SPARSITY = 0.01
X_train, y_train, X_test, y_test = create_torch_XOR_XNOR_dataset()

EVALUTION_RUNS = 5
THRESHOLD = 0.05

# Counter for 100% accuracy runs
perfect_accuracy_count = 0
evaluation_results = []

for _ in range(EVALUTION_RUNS):
    model_3_layer_MLP_XNOR_XOR = SparseThreeLayerMLP().double()

    # define the loss
    loss_function = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model_3_layer_MLP_XNOR_XOR.parameters(), lr=INIITIAL_LEARNING_RATE, eps=1e-7)

    all_loss = train_with_rectified_L2(model_3_layer_MLP_XNOR_XOR, 
                                    loss_function, 
                                    optimizer, 
                                    X_train, 
                                    y_train,
                                    no_of_epochs=EPOCHS,
                                    ALPHA=ALPHA,
                                    LAMBDA_MAGNITUDE=LAMBDA_MAGNITUDE,
                                    LAMBDA_POLARITY=LAMBDA_POLARITY,
                                    LAMBDA_INTEGERS=LAMBDA_INTEGERS,
                                    initial_lr=INIITIAL_LEARNING_RATE,
                                    max_lr=MAXIMUM_LEARNING_RATE)

    # Evaluate accuracy for the function (A xor B) and (C xnor D)
    inputs = [
        torch.tensor([[0.0, 0.0, 0.0, 0.0]], dtype=torch.float64),
        torch.tensor([[0.0, 0.0, 0.0, 1.0]], dtype=torch.float64),
        torch.tensor([[0.0, 0.0, 1.0, 0.0]], dtype=torch.float64),
        torch.tensor([[0.0, 0.0, 1.0, 1.0]], dtype=torch.float64),
        torch.tensor([[0.0, 1.0, 0.0, 0.0]], dtype=torch.float64),
        torch.tensor([[0.0, 1.0, 0.0, 1.0]], dtype=torch.float64),
        torch.tensor([[0.0, 1.0, 1.0, 0.0]], dtype=torch.float64),
        torch.tensor([[0.0, 1.0, 1.0, 1.0]], dtype=torch.float64),
        torch.tensor([[1.0, 0.0, 0.0, 0.0]], dtype=torch.float64),
        torch.tensor([[1.0, 0.0, 0.0, 1.0]], dtype=torch.float64),
        torch.tensor([[1.0, 0.0, 1.0, 0.0]], dtype=torch.float64),
        torch.tensor([[1.0, 0.0, 1.0, 1.0]], dtype=torch.float64),
        torch.tensor([[1.0, 1.0, 0.0, 0.0]], dtype=torch.float64),
        torch.tensor([[1.0, 1.0, 0.0, 1.0]], dtype=torch.float64),
        torch.tensor([[1.0, 1.0, 1.0, 0.0]], dtype=torch.float64),
        torch.tensor([[1.0, 1.0, 1.0, 1.0]], dtype=torch.float64)
    ]
    expected_outputs = [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
    correct_predictions = 0

    for input_tensor, expected_output in zip(inputs, expected_outputs):
        model_output = model_3_layer_MLP_XNOR_XOR(input_tensor).item()
        if abs(model_output - expected_output) <= THRESHOLD:
            correct_predictions += 1

    accuracy = correct_predictions / len(expected_outputs)

    if accuracy > 0.95:
        perfect_accuracy_count += 1

    # Evaluate interpretability
    results = evaluate_logic_gates_in_network(model_3_layer_MLP_XNOR_XOR, 2)
    results['accuracy'] = accuracy

    evaluation_results.append(results)

# Save the results to a JSON file
with open('evaluation_results.json', 'w') as f:
    json.dump(evaluation_results, f, indent=4)

In [3]:
import json

# Load the JSON data from a file
def load_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

# Calculate the average accuracy and percentage
def calculate_averages(data):
    total_accuracy = 0
    total_percentage = 0
    count = len(data)
    
    for record in data:
        total_accuracy += record["accuracy"]
        total_percentage += record["percentage"]
    
    average_accuracy = total_accuracy / count if count != 0 else 0
    average_percentage = total_percentage / count if count != 0 else 0
    
    return average_accuracy, average_percentage


filename = 'evaluation_results.json'  # Replace with your actual JSON file name
data = load_json(filename)
average_accuracy, average_percentage = calculate_averages(data)

print(f"Average Accuracy: {average_accuracy}")
print(f"Average Percentage: {average_percentage}")

Average Accuracy: 0.2875
Average Percentage: 52.857142857142854
